# OAI RAN Bug Root-Cause Analysis Workflow

This notebook implements a structured workflow for investigating bugs in the OpenAirInterface5G (OAI) RAN project. It follows the process defined in `QAprompt.json`, guiding the user from raw bug reports to root-cause analysis by cross-referencing specifications and code.

### Agent Instruction

If you are a AI coding agent, you are given access to access the entire codebase in the current folder. Also, you have access to "spec_ingestor.py" to help you make correct citation to the latest 3GPP standard. Note it is not using AI and you should consider it a grep tool with structured output. If you make any claim about 3GPP standard, you *MUST* verify it with this tool. Use it strictly follow this pattern "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/spec_ingestor.py F1AP.docx 'keyword'" for F1AP and "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/spec_ingestor.py NGAP.docx 'keyword'"

### Global Definitions
The workflow adheres to the following definitions and enumerations:

```json
  "$defs": {
    "RepoEnum": {
      "type": "string",
      "enum": ["https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/openair2/F1AP?ref_type=heads"]
    },
    "InterfaceEnum": {
      "type": "string",
      "enum": ["F1AP","E1AP","NGAP","XnAP","RRC","PDCP","RLC","MAC","GTP-U","SCTP"]
    },
    "VerdictEnum": {
      "type": "string",
      "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"]
    },
    "ComponentEnum": {
      "type": "string",
      "enum": ["gNB-CU", "gNB-DU", "UE", "AMF", "UPF"]
    }
  },
```

The workflow consists of sequential steps, each broken down into:
1. **Requirements:** Definition of the step's goal, inputs, and expected outputs based on `QAprompt.json`.
2. **Results:** Placeholders for the execution results.
3. **Check:** Verification criteria to ensure the step was completed successfully.

## Step 1: Bug Ingest

### 1.1 Requirements

**Goal:** Normalize a freeform bug report into a structured bug card with interface/procedure guesses and key identifiers.

**Input:**
- `bug_text`: Bug: gNB-DU bypasses F1AP Setup procedure when receiving UEContextModificationRequest, and starts gtpu instance directly.
Repro Steps:
1) Attacker sends an incorrect UE Context Setup Request to the gNB-CU.
2) gNB-CU rejects the request as expected.
3) Attacker immediately sends a UE Context Setup Success.
4) In parallel, attacker sends an initialRRCTransfer with mutated DU ID and Transaction ID.
5) Attacker follows with an RRC Setup Success message.

**Role & Instructions:**
Act as **BugCardBuilder**. Extract and normalize from `bug_text`: likely interface(s), procedure(s), component roles, and key IDs (transaction IDs, CU/DU UE F1AP IDs, DU ID, RNTI, PCI, served-cell list). Summarize observed vs expected (if implied).

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["bug_card"],
        "properties": {
          "bug_card": {
            "type": "object",
            "required": ["observed_behavior", "interface_guess", "procedure_guess", "components_involved"],
            "properties": {
              "observed_behavior": { "type": "string" },
              "interface_guess": { "type": "array", "items": {"$ref":"#/$defs/InterfaceEnum" } },
              "procedure_guess": { "type": "array", "items": { "type": "string" } },
              "components_involved": { "type": "array", "items": {"$ref":"#/$defs/ComponentEnum" },"minItems":1 },
              "key_ids": {
                "type": "object",
                "properties": {
                  "transaction_id": { "type": "string" },
                  "cu_ue_f1ap_id": { "type": "string" },
                  "du_ue_f1ap_id": { "type": "string" },
                  "du_id": { "type": "string" },
                  "rnti": { "type": "string" },
                  "pci": { "type": "string" },
                  "served_cell_list": { "type": "array", "items": { "type": "string" } }
                }
              },
              "signals_or_timers": { "type": "array", "items": { "type": "string" } }
            }
          }
        }
      }
```

```json
{
  "bug_card": {
    "observed_behavior": "gNB-DU allows bypassing the F1AP UE Context Setup procedure by accepting a UE Context Modification Request for a UE that has not yet completed the Setup procedure, leading to premature GTP-U instance startup.",
    "interface_guess": [
      "F1AP",
      "RRC",
      "GTP-U"
    ],
    "procedure_guess": [
      "UE Context Setup",
      "UE Context Modification",
      "Initial UL RRC Message Transfer"
    ],
    "components_involved": [
      "gNB-DU",
      "gNB-CU"
    ],
    "key_ids": {
      "du_id": "mutated DU ID",
      "transaction_id": "mutated Transaction ID"
    },
    "signals_or_timers": [
      "UE Context Setup Request",
      "UE Context Setup Success",
      "UEContextModificationRequest",
      "initialRRCTransfer",
      "RRC Setup Success"
    ]
  }
}
```

### 1.3 Check

**Verification:** Check that the output matches the JSON schema, specifically that `interface_guess` and `procedure_guess` are not empty, and no freeform prose is included.

## Step 2: Spec Section Fetcher

### 2.1 Requirements

**Goal:** Propose spec sections (procedures, timers, message formats) that likely govern the bug.

**Input:**
- `bug_card`: Structured description from Step 1.
- `spec_toc`: Table of contents with ids/titles/anchors.

**Role & Instructions:**
Act as **SpecIndexer**. Using `bug_card` and attached spec slices, identify 3–6 most relevant spec sections. Produce keywords, titles, and summarize `expected_behaviour`.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_spec_sections","expected_behaviour"],
        "properties": {
          "candidate_spec_sections": {
            "type": "array",
            "minItems": 3,
            "maxItems": 6,
            "items": {
              "type": "object",
              "required": ["Keywords", "title"],
              "properties": { "Keywords": { "type": "array", "items": { "type": "string" }}, "title": { "type": "array", "items": { "type": "string" }} }
            }
          },
          "expected_behaviour": {"type": "string"}
        }
      }
```

```json
{
  "candidate_spec_sections": [
    {
      "Keywords": [
        "UE Context Setup"
      ],
      "title": [
        "8.3.1",
        "UE Context Setup"
      ]
    },
    {
      "Keywords": [
        "UE Context Modification",
        "established UE Context"
      ],
      "title": [
        "8.3.4",
        "UE Context Modification"
      ]
    },
    {
      "Keywords": [
        "Initial UL RRC Message Transfer"
      ],
      "title": [
        "8.4.1",
        "Initial UL RRC Message Transfer"
      ]
    }
  ],
  "expected_behaviour": "The UE Context Modification procedure is intended to modify an *established* UE context. The gNB-DU should reject a Modification Request if the UE context has not been successfully established via the UE Context Setup procedure."
}
```

### 2.3 Check

**Verification:** Check that 3-6 relevant sections are identified and `expected_behaviour` is summarized.

## Step 3: Code Fetcher (metadata-driven)

### 3.1 Requirements

**Goal:** Select likely source files/functions using metadata (paths, interface names, keywords, ASN.1 types, timers).

**Input:**
- `bug_card`: From Step 1.
- `candidate_spec_sections`: From Step 2.
- `repo_metadata`: Path, interface names, and known keywords.

**Role & Instructions:**
Act as **CodeLocator**. Build a prioritized list of files/functions to fetch using interface names, procedure/timer, and keywords. Plan recursion if initial fetch seems insufficient.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_code"],
        "properties": {
          "candidate_code": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["path", "function_name", "reason"],
              "properties": { "path": { "type": "string" }, "function_name": { "type": "string" }, "reason": { "type": "string" } }
            }
          }
        }
      }
```

```json
{
  "candidate_code": [
    {
      "path": "openair2/LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c",
      "function_name": "ue_context_modification_request",
      "reason": "Core logic handling UE Context Modification Request in DU/MAC. Lacks check for established context."
    },
    {
      "path": "openair2/F1AP/f1ap_du_ue_context_management.c",
      "function_name": "DU_handle_UE_CONTEXT_MODIFICATION_REQUEST",
      "reason": "F1AP handler that decodes the request and calls the MAC layer handler."
    },
    {
      "path": "openair2/LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c",
      "function_name": "handle_ue_context_drbs_setup",
      "reason": "Function that creates GTP-U tunnels without verifying if the UE context is fully established."
    }
  ]
}
```

### 3.3 Check

**Verification:** Check that `candidate_code` is not empty and includes paths and function names.

## Step 4: Lightweight Retriever

### 4.1 Requirements

**Goal:** Retrieve high-signal snippets only from shortlisted specs and code.

**Input:**
- `candidate_spec_sections`: From Step 2.
- `candidate_code`: From Step 3.

**Role & Instructions:**
Act as **SnippetRetriever**. For each query, return top snippets with exact locations (spec id/anchor, file:line-range). Keep each snippet ≤ 120 words.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["snippets"],
        "properties": {
          "snippets": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["source", "location", "text", "kind"],
              "properties": {
                "kind": { "type": "string", "enum": ["spec", "code"] },
                "source": { "type": "string" },
                "location": { "type": "string" },
                "text": { "type": "string" }
              }
            }
          }
        }
      }
```

```json
{
  "snippets": [
    {
      "kind": "spec",
      "source": "TS 38.473",
      "location": "8.3.4.1",
      "text": "The purpose of the UE Context Modification procedure is to modify the established UE Context..."
    },
    {
      "kind": "code",
      "source": "openair2/LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c",
      "location": "742:747",
      "text": "NR_UE_info_t *UE = find_nr_UE(&RC.nrmac[0]->UE_info, req->gNB_DU_ue_id);\\nif (!UE) {\\n  LOG_E(NR_MAC, \"could not find UE with RNTI %04x\\n\", req->gNB_DU_ue_id);\\n  NR_SCHED_UNLOCK(&mac->sched_lock);\\n  return;\\n}"
    }
  ]
}
```

### 4.3 Check

**Verification:** Check that snippets are retrieved from both spec and code, with exact locations.

## Step 5: Event/State Sketcher (draft)

### 5.1 Requirements

**Goal:** Draft expected message sequence and 5–8 state sketch from snippets.

**Input:**
- `bug_card`: From Step 1.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **SketchMaker**. Build (1) an expected message sequence as ordered events and (2) a compact state machine (5–8 states, transitions with guards). Note assumptions and open questions.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["sequence_diagram", "state_machine", "assumptions", "questions", "review_request"],
        "properties": {
          "sequence_diagram": {
            "type": "array",
            "items": { "type": "object", "required": ["from", "to", "message"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "message": { "type": "string" }, "precond": { "type": "string" }, "postcond": { "type": "string" } } }
          },
          "state_machine": {
            "type": "object",
            "required": ["states", "transitions"],
            "properties": {
              "states": { "type": "array", "items": { "type": "string" }, "minItems": 5, "maxItems": 8 },
              "transitions": {
                "type": "array",
                "items": { "type": "object", "required": ["from", "to", "on"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "on": { "type": "string" }, "guard": { "type": "string" } } }
              }
            }
          },
          "assumptions": { "type": "array", "items": { "type": "string" } },
          "questions": { "type": "array", "items": { "type": "string" } },
          "review_request": { "type": "string", "enum": ["HumanInLoop_B_required"] }
        }
      }
```

```json
{
  "sequence_diagram": [
    {
      "from": "UE",
      "to": "gNB-DU",
      "message": "RRCSetupRequest (Msg3)"
    },
    {
      "from": "gNB-DU",
      "to": "gNB-CU",
      "message": "INITIAL UL RRC MESSAGE TRANSFER",
      "postcond": "UE State: RRC_CONNECTED_PENDING"
    },
    {
      "from": "gNB-CU",
      "to": "gNB-DU",
      "message": "UE CONTEXT MODIFICATION REQUEST",
      "precond": "UE Context Not Established"
    },
    {
      "from": "gNB-DU",
      "to": "gNB-DU",
      "message": "ue_context_modification_request",
      "postcond": "GTP-U Tunnels Created (Bug)"
    },
    {
      "from": "gNB-DU",
      "to": "gNB-CU",
      "message": "UE CONTEXT MODIFICATION RESPONSE"
    }
  ],
  "state_machine": {
    "states": [
      "NULL",
      "RRC_CONNECTED_PENDING",
      "ESTABLISHED"
    ],
    "transitions": [
      {
        "from": "NULL",
        "to": "RRC_CONNECTED_PENDING",
        "on": "Msg3"
      },
      {
        "from": "RRC_CONNECTED_PENDING",
        "to": "ESTABLISHED",
        "on": "UE CONTEXT SETUP REQUEST"
      },
      {
        "from": "ESTABLISHED",
        "to": "ESTABLISHED",
        "on": "UE CONTEXT MODIFICATION REQUEST"
      },
      {
        "from": "RRC_CONNECTED_PENDING",
        "to": "ESTABLISHED",
        "on": "UE CONTEXT MODIFICATION REQUEST",
        "guard": "Missing Guard (Bug)"
      }
    ]
  },
  "assumptions": [
    "The attacker can act as a malicious CU or inject F1AP messages."
  ],
  "questions": [
    "Does the CU accept the Modification Response for a UE it didn't setup?"
  ],
  "review_request": "HumanInLoop_B_required"
}
```

### 5.3 Check

**Verification:** Check for a sequence diagram, a state machine with 5-8 states, and any assumptions.

## Step 6: Conformance Checker (simple invariants)

### 6.1 Requirements

**Goal:** Check core invariants to propose preliminary verdict.

**Input:**
- `bug_card`: From Step 1.
- `sequence_diagram`: From Step 5.
- `state_machine`: From Step 5.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **ConformanceChecker**. Using snippets as authority, test invariants: directionality, req/resp pairing, ID binding, transaction scoping, timer preconditions. For each invariant: PASS/FAIL with cite. Propose preliminary verdict.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["invariant_checks", "preliminary_verdict", "rationale"],
        "properties": {
          "invariant_checks": {
            "type": "array",
            "items": { "type": "object", "required": ["name", "result", "evidence"], "properties": { "name": { "type": "string" }, "result": { "type": "string", "enum": ["PASS", "FAIL", "UNKNOWN"] }, "evidence": { "type": "array", "items": { "type": "string" } } } }
          },
          "preliminary_verdict": { "type": "string", "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"] },
          "rationale": { "type": "string" }
        }
      }
```

```json
{
  "invariant_checks": [
    {
      "name": "Modification Requires Established Context",
      "result": "FAIL",
      "evidence": [
        "Spec 8.3.4.1 requires 'established UE Context'",
        "Code in mac_rrc_dl_handler.c allows Modification in any state where UE exists"
      ]
    }
  ],
  "preliminary_verdict": "implementation-bug",
  "rationale": "The implementation fails to verify the UE context state before processing UE Context Modification Request, allowing a bypass of the Setup procedure."
}
```

### 6.3 Check

**Verification:** Check that invariants are tested and a preliminary verdict is proposed.

## Step 7: Evidence Linker

### 7.1 Requirements

**Goal:** Bundle a compact evidence pack linking spec clauses and code lines with an expected-vs-observed chain.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **EvidenceBundler**. Assemble a minimal yet complete pack: spec refs (id/anchor), code refs (file:lines), and a single paragraph contrasting expected vs observed.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["evidence_pack"],
        "properties": {
          "evidence_pack": {
            "type": "object",
            "required": ["spec_citations", "code_citations", "expected_vs_observed"],
            "properties": {
              "spec_citations": { "type": "array", "items": { "type": "string" } },
              "code_citations": { "type": "array", "items": { "type": "string" } },
              "expected_vs_observed": { "type": "string" }
            }
          }
        }
      }
```

```json
{
  "evidence_pack": {
    "spec_citations": [
      "TS 38.473 8.3.4.1: 'modify the established UE Context'"
    ],
    "code_citations": [
      "openair2/LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c:724"
    ],
    "expected_vs_observed": "Expected: gNB-DU should reject UE Context Modification Request if the UE context is not in ESTABLISHED state (i.e. before UE Context Setup). Observed: gNB-DU processes the request and creates GTP tunnels as long as the UE RNTI exists."
  }
}
```

### 7.3 Check

**Verification:** Check that an evidence pack with spec and code citations and a comparison paragraph is created.

## Step 8: Report Generator

### 8.1 Requirements

**Goal:** Emit a one-page final report and request human sign-off on verdict.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `evidence_pack`: From Step 7.

**Role & Instructions:**
Act as **ReportWriter**. Produce the final JSON report with {verdict, rationale, citations, suggested_repro, risks, next_steps}. Be concise and actionable.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["report", "review_request"],
        "properties": {
          "report": {
            "type": "object",
            "required": ["verdict", "rationale", "citations", "suggested_repro", "next_steps"],
            "properties": {
              "verdict": { "type": "string", "$ref": "#/$defs/VerdictEnum" },
              "rationale": { "type": "string" },
              "citations": {
                "type": "object",
                "properties": {
                  "spec": { "type": "array", "items": { "type": "string" } },
                  "code": { "type": "array", "items": { "type": "string" } }
                }
              },
              "suggested_repro": { "type": "array", "items": { "type": "string" } },
              "risks": { "type": "array", "items": { "type": "string" } },
              "next_steps": { "type": "array", "items": { "type": "string" } }
            }
          },
          "review_request": { "type": "string", "enum": ["HumanInLoop_C_required"] }
        }
      }
```

```json
{
  "report": {
    "verdict": "implementation-bug",
    "rationale": "gNB-DU processes UE Context Modification Request without checking if the UE context is established, violating TS 38.473 8.3.4.1. This allows bypassing security context setup and creating GTP tunnels prematurely.",
    "citations": {
      "spec": [
        "TS 38.473 8.3.4.1"
      ],
      "code": [
        "openair2/LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c"
      ]
    },
    "suggested_repro": [
      "Send Initial UL RRC Message",
      "Send UE Context Modification Request (without Setup)"
    ],
    "risks": [
      "Unauthenticated traffic injection",
      "State confusion"
    ],
    "next_steps": [
      "Add state check in ue_context_modification_request"
    ]
  },
  "review_request": "HumanInLoop_C_required"
}
```

### 8.3 Check

**Verification:** Check that the final report contains a verdict, rationale, citations, and next steps.